
<img align="left" src = https://www.linea.gov.br/wp-content/themes/LIneA/imagens/logo-header.png width=120 style="padding: 20px"> <br> 

# Photo-z Parsl Afterburner
Last verified run: 07/27/2022


Disclaimer: preliminary version

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yaml

%matplotlib inline 

#### Hardcoded paths (temporary)

In [ ]:
config_path = "../sample-data/sample.yml"
input_paths = ["../sample-data/cats/part-a.parquet",
               "../sample-data/cats/part-b.parquet",
               "../sample-data/cats/part-c.parquet",
               "../sample-data/cats/part-d.parquet",
               "../sample-data/cats/part-e.parquet"
              ]
output_paths = ["../sandbox/data/outputs/part-a/photz-00003.parquet",
              "../sandbox/data/outputs/part-b/photz-00001.parquet",
              "../sandbox/data/outputs/part-c/photz-00002.parquet",
              "../sandbox/data/outputs/part-d/photz-00005.parquet",
              "../sandbox/data/outputs/part-e/photz-00004.parquet"
             ]

To do: make output paths consistent (alphabetical/numeric)

### Configuration used

In [ ]:
with open(config_path) as _file:
    phz_config = yaml.load(_file, Loader=yaml.FullLoader)
    settings = phz_config.get('settings', {})
settings

### Input data 

In [ ]:
data = pd.read_parquet(input_paths[0])
for i in range(1,5):
    data = pd.concat([data, pd.read_parquet(input_paths[i])])

In [ ]:
data.head()

In [ ]:
data.describe()

#### Spatial distribution

In [ ]:
plt.figure()
plt.hexbin(data.RA, data.DEC, gridsize=50, mincnt=1) 
plt.xlabel("R.A.", fontsize=14)
plt.ylabel("Dec.", fontsize=14)
plt.colorbar(label="density of points")
plt.tight_layout()

### Magnitudes

In [ ]:
from utils import get_photometric_columns
columns_list = get_photometric_columns(settings['bands'], settings['photo_type'], 
                                       settings['err_type'], settings['index'])
columns_list

In [ ]:
data.columns

In [ ]:
plt.figure()
for band in settings['bands']:
    mag = settings['photo_type'].format(band)
    query = mag + "< 40."
    plt.hist(data.query(query)[mag], histtype="step", bins=np.arange(15, 30, 0.4), label=band, lw=2)
plt.legend()
plt.xlabel("magnitude")
plt.xlim(15, 30)
plt.tight_layout()

In [ ]:
for band in settings['bands']:
    plt.figure()
    plt.plot(data[settings['photo_type'].format(band)],  
             data[settings['err_type'].format(band)], 
             ".", alpha=0.5)
    plt.xlim(15,30)
    plt.ylim(0,2)
    plt.xlabel(settings['photo_type'].format(band))    
    plt.ylabel(settings['err_type'].format(band))
    plt.tight_layout()

### Outputs  (LePhare results)

In [ ]:
data_a = pd.read_parquet(output_paths[0])
data_b = pd.read_parquet(output_paths[1])
data_c = pd.read_parquet(output_paths[2])
data_d = pd.read_parquet(output_paths[3])
data_e = pd.read_parquet(output_paths[4])

In [ ]:
photoz = pd.read_parquet(output_paths[0])
for i in range(1,5):
    photoz = pd.concat([photoz, pd.read_parquet(output_paths[i])])

In [ ]:
photoz.head()

In [ ]:
photoz.describe()

### Redshift distribution 

#### Best fit photo-z

In [ ]:
plt.figure()
plt.hist(photoz.z_best, histtype="step", label="z_best", lw=2,
         bins=np.arange(0, 2, 0.05))
plt.xlabel("photo-z")
plt.ylabel("counts")
plt.legend()
plt.tight_layout()

#### Maximum likelihood photo-z

In [ ]:
plt.figure()
plt.hist(photoz.z_ml, histtype="step", label="z_ml", lw=2,
         bins=np.arange(0, 2, 0.05))
plt.xlabel("photo-z")
plt.ylabel("counts")
plt.legend()
plt.tight_layout()

#### Photo-z errors

In [ ]:
plt.figure()
plt.hist(photoz.err_z, histtype="step", lw=2,
         bins=np.arange(0, 1, 0.02))
plt.xlabel("photo-z error")
plt.ylabel("counts")
plt.tight_layout()

#### Best fit versus maximum likelihood photo-zs

In [ ]:
plt.figure(figsize=[6,6])
plt.plot(photoz.z_best, photoz.z_ml, ".")
plt.plot([-1, 3], [-1, 3], 'r-')
plt.xlabel("photo-z (z_best)")
plt.ylabel("photo-z (z_ml)")
plt.xlim(0., 2.)
plt.ylim(0., 2.)
plt.tight_layout()